In [1]:
import torch
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import numpy as np
import torch.nn as nn
from math import *

In [2]:
torch.cuda.set_device(0)

In [3]:
torch.set_default_tensor_type('torch.DoubleTensor') # 设置浮点类型为 torch.float64

In [4]:
# 定义激活函数: swish(x)
def acti(x):
    return x*torch.sigmoid(x)  

In [5]:
# 定义网络结构
class DeepRitzNet(torch.nn.Module):
    def __init__(self, input_width, layer_width):
        super(DeepRitzNet, self).__init__()
        self.linear_in = torch.nn.Linear(input_width, layer_width)
        self.linear1 = torch.nn.Linear(layer_width, layer_width)
        self.linear2 = torch.nn.Linear(layer_width, layer_width)
        self.linear3 = torch.nn.Linear(layer_width, layer_width)
        self.linear4 = torch.nn.Linear(layer_width, layer_width)
        self.linear5 = torch.nn.Linear(layer_width, layer_width)
        self.linear6 = torch.nn.Linear(layer_width, layer_width)
        self.linear_out = torch.nn.Linear(layer_width, 1)

    def forward(self, x):
        y = self.linear_in(x) # fully connect layer
        y = y + acti(self.linear2(acti(self.linear1(y)))) # residual block 1
        y = y + acti(self.linear4(acti(self.linear3(y)))) # residual block 2
        y = y + acti(self.linear6(acti(self.linear5(y)))) # residual block 3
        output = self.linear_out(y) # fully connect layer
        return output

In [6]:
dimension = 8

In [7]:
# exact solution
def u_ex(x):  
    x_temp = torch.cos(pi*x)
    u_x = (x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return u_x

In [8]:
def f(x):
    x_temp = torch.cos(pi*x)
    f_x = 2*pi**2*(x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return f_x

In [9]:
def DRM(x):
    u_hat = model(x)
    uxx = torch.zeros(x.size()[0], dimension).cuda() 
    step_size = 0.0001
    for i in range(dimension):
        dx = torch.zeros(x.size()[0], dimension).cuda() 
        dx[:, i] = torch.ones(x.size()[0])
        uxx[:, i] = (model(x+step_size*dx) - 2*model(x) + model(x-step_size*dx))[:,0]/step_size**2
    laplace_u = (torch.sum(uxx, dim = 1)).reshape([x.size()[0], 1]) # dim = 1 按行求和
    f_temp = f(x)
    part_1 = torch.sum((-laplace_u + pi**2*u_hat - f_temp)**2)/x.size()[0]
    
    Nb = 100
    xa1 = torch.rand(Nb, dimension).cuda()  
    xa1[:, 0] = torch.zeros(Nb)
    xa2 = torch.rand(Nb, dimension).cuda()  
    xa2[:, 0] = torch.ones(Nb)
    xb1 = torch.rand(Nb, dimension).cuda() 
    xb1[:, 1] = torch.zeros(Nb)
    xb2 = torch.rand(Nb, dimension).cuda() 
    xb2[:, 1] = torch.ones(Nb)
    xc1 = torch.rand(Nb, dimension).cuda() 
    xc1[:, 2] = torch.zeros(Nb)
    xc2 = torch.rand(Nb, dimension).cuda() 
    xc2[:, 2] = torch.ones(Nb)
    xd1 = torch.rand(Nb, dimension).cuda() 
    xd1[:, 3] = torch.zeros(Nb)
    xd2 = torch.rand(Nb, dimension).cuda() 
    xd2[:, 3] = torch.ones(Nb)
    xe1 = torch.rand(Nb, dimension).cuda()  
    xe1[:, 4] = torch.zeros(Nb)
    xe2 = torch.rand(Nb, dimension).cuda()  
    xe2[:, 4] = torch.ones(Nb)
    xf1 = torch.rand(Nb, dimension).cuda() 
    xf1[:, 5] = torch.zeros(Nb)
    xf2 = torch.rand(Nb, dimension).cuda() 
    xf2[:, 5] = torch.ones(Nb)
    xg1 = torch.rand(Nb, dimension).cuda() 
    xg1[:, 6] = torch.zeros(Nb)
    xg2 = torch.rand(Nb, dimension).cuda() 
    xg2[:, 6] = torch.ones(Nb)
    xh1 = torch.rand(Nb, dimension).cuda() 
    xh1[:, 7] = torch.zeros(Nb)
    xh2 = torch.rand(Nb, dimension).cuda() 
    xh2[:, 7] = torch.ones(Nb)
    
    dx1 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx1[:, 0] = torch.ones(xb1.size()[0])
    dx2 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx2[:, 1] = torch.ones(xb1.size()[0])
    dx3 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx3[:, 2] = torch.ones(xb1.size()[0])
    dx4 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx4[:, 3] = torch.ones(xb1.size()[0])
    dx5 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx5[:, 4] = torch.ones(xb1.size()[0])
    dx6 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx6[:, 5] = torch.ones(xb1.size()[0])
    dx7 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx7[:, 6] = torch.ones(xb1.size()[0])
    dx8 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx8[:, 7] = torch.ones(xb1.size()[0])
    
    uxa1 = (((model(xa1+step_size*dx1) - model(xa1-step_size*dx1))/step_size/2 * (1.0) - 0)**2).cuda() 
    uxa2 = (((model(xa2+step_size*dx1) - model(xa2-step_size*dx1))/step_size/2 * (-1.0) - 0)**2).cuda() 
    uxb1 = (((model(xb1+step_size*dx2) - model(xb1-step_size*dx2))/step_size/2 * (1.0) - 0)**2).cuda() 
    uxb2 = (((model(xb2+step_size*dx2) - model(xb2-step_size*dx2))/step_size/2 * (-1.0) - 0)**2).cuda() 
    uxc1 = (((model(xc1+step_size*dx3) - model(xc1-step_size*dx3))/step_size/2 * (1.0) - 0)**2).cuda() 
    uxc2 = (((model(xc2+step_size*dx3) - model(xc2-step_size*dx3))/step_size/2 * (-1.0) - 0)**2).cuda() 
    uxd1 = (((model(xd1+step_size*dx4) - model(xd1-step_size*dx4))/step_size/2 * (1.0) - 0)**2).cuda() 
    uxd2 = (((model(xd2+step_size*dx4) - model(xd2-step_size*dx4))/step_size/2 * (-1.0) - 0)**2).cuda() 
    uxe1 = (((model(xe1+step_size*dx5) - model(xe1-step_size*dx5))/step_size/2 * (1.0) - 0)**2).cuda() 
    uxe2 = (((model(xe2+step_size*dx5) - model(xe2-step_size*dx5))/step_size/2 * (-1.0) - 0)**2).cuda() 
    uxf1 = (((model(xf1+step_size*dx6) - model(xf1-step_size*dx6))/step_size/2 * (1.0) - 0)**2).cuda() 
    uxf2 = (((model(xf2+step_size*dx6) - model(xf2-step_size*dx6))/step_size/2 * (-1.0) - 0)**2).cuda() 
    uxg1 = (((model(xg1+step_size*dx7) - model(xg1-step_size*dx7))/step_size/2 * (1.0) - 0)**2).cuda() 
    uxg2 = (((model(xg2+step_size*dx7) - model(xg2-step_size*dx7))/step_size/2 * (-1.0) - 0)**2).cuda() 
    uxh1 = (((model(xh1+step_size*dx8) - model(xh1-step_size*dx8))/step_size/2 * (1.0) - 0)**2).cuda() 
    uxh2 = (((model(xh2+step_size*dx8) - model(xh2-step_size*dx8))/step_size/2 * (-1.0) - 0)**2).cuda() 
    
    part_2 = torch.sum((uxa1 + uxa2 + uxb1 + uxb2 + uxc1 + uxc2 + uxd1 + uxd2 + uxe1 + uxe2 + uxf1 + uxf2 + uxg1 + uxg2 + uxh1 + uxh2)[:,0])/xa1.size()[0]
    lambda1 = 1.0
    return part_1 + lambda1 * part_2 / 16

In [10]:
Data_size = 2000
def Gendata():
    x = torch.rand(Data_size, dimension)
    return x.cuda()

In [11]:
# 正态分布初始化参数
def initparam(model, sigma):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            m.weight.data.normal_(0, sigma)
    return model

In [12]:
model = DeepRitzNet(dimension, 16)
model = initparam(model, 0.5)

In [13]:
device = torch.device("cuda:0" )
model.to(device)

DeepRitzNet(
  (linear_in): Linear(in_features=8, out_features=16, bias=True)
  (linear1): Linear(in_features=16, out_features=16, bias=True)
  (linear2): Linear(in_features=16, out_features=16, bias=True)
  (linear3): Linear(in_features=16, out_features=16, bias=True)
  (linear4): Linear(in_features=16, out_features=16, bias=True)
  (linear5): Linear(in_features=16, out_features=16, bias=True)
  (linear6): Linear(in_features=16, out_features=16, bias=True)
  (linear_out): Linear(in_features=16, out_features=1, bias=True)
)

In [14]:
import torch.optim as optim
import torch.nn as nn
import time

In [15]:
def relative_error():
    x = Gendata()
    predict = model(x)
    exact = u_ex(x)
    value = torch.sqrt(torch.sum((predict - exact )**2))/torch.sqrt(torch.sum((exact )**2))
    return value

In [16]:
traintime = 50000
error_save = np.zeros(traintime)
optimizer = optim.Adam(model.parameters())

In [17]:
time_start = time.time()
for i in range(traintime):
    optimizer.zero_grad()
    x = Gendata()
    x.requires_grad = True
    losses = DRM(x)
    losses.backward()
    optimizer.step()
    error = relative_error()
    error_save[i] = float(error)
    
    if i % 50 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", losses.detach())
        print("current relative error is: ", error.detach())
        np.save("DGM_relative_error_8D_Neumann.npy", error_save)
np.save("DGM_relative_error_8D_Neumann.npy", error_save)
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

current epoch is:  0
current loss is:  tensor(26322.1748, device='cuda:0')
current relative error is:  tensor(2.1807, device='cuda:0')
current epoch is:  50
current loss is:  tensor(2414.6890, device='cuda:0')
current relative error is:  tensor(1.3251, device='cuda:0')
current epoch is:  100
current loss is:  tensor(1324.7389, device='cuda:0')
current relative error is:  tensor(0.9920, device='cuda:0')
current epoch is:  150
current loss is:  tensor(658.2921, device='cuda:0')
current relative error is:  tensor(0.8421, device='cuda:0')
current epoch is:  200
current loss is:  tensor(337.6621, device='cuda:0')
current relative error is:  tensor(0.7495, device='cuda:0')
current epoch is:  250
current loss is:  tensor(212.8374, device='cuda:0')
current relative error is:  tensor(0.7483, device='cuda:0')
current epoch is:  300
current loss is:  tensor(170.8519, device='cuda:0')
current relative error is:  tensor(0.7455, device='cuda:0')
current epoch is:  350
current loss is:  tensor(134.25

current epoch is:  3050
current loss is:  tensor(30.4732, device='cuda:0')
current relative error is:  tensor(0.8067, device='cuda:0')
current epoch is:  3100
current loss is:  tensor(30.4962, device='cuda:0')
current relative error is:  tensor(0.7948, device='cuda:0')
current epoch is:  3150
current loss is:  tensor(29.3719, device='cuda:0')
current relative error is:  tensor(0.7670, device='cuda:0')
current epoch is:  3200
current loss is:  tensor(28.3881, device='cuda:0')
current relative error is:  tensor(0.7766, device='cuda:0')
current epoch is:  3250
current loss is:  tensor(29.4708, device='cuda:0')
current relative error is:  tensor(0.8023, device='cuda:0')
current epoch is:  3300
current loss is:  tensor(28.7866, device='cuda:0')
current relative error is:  tensor(0.7804, device='cuda:0')
current epoch is:  3350
current loss is:  tensor(28.1109, device='cuda:0')
current relative error is:  tensor(0.8334, device='cuda:0')
current epoch is:  3400
current loss is:  tensor(28.081

current epoch is:  6100
current loss is:  tensor(13.4816, device='cuda:0')
current relative error is:  tensor(0.6264, device='cuda:0')
current epoch is:  6150
current loss is:  tensor(15.1256, device='cuda:0')
current relative error is:  tensor(0.7002, device='cuda:0')
current epoch is:  6200
current loss is:  tensor(14.8769, device='cuda:0')
current relative error is:  tensor(0.6835, device='cuda:0')
current epoch is:  6250
current loss is:  tensor(13.4979, device='cuda:0')
current relative error is:  tensor(0.6771, device='cuda:0')
current epoch is:  6300
current loss is:  tensor(12.9976, device='cuda:0')
current relative error is:  tensor(0.6585, device='cuda:0')
current epoch is:  6350
current loss is:  tensor(12.7637, device='cuda:0')
current relative error is:  tensor(0.6432, device='cuda:0')
current epoch is:  6400
current loss is:  tensor(13.4505, device='cuda:0')
current relative error is:  tensor(0.6741, device='cuda:0')
current epoch is:  6450
current loss is:  tensor(11.814

current epoch is:  9200
current loss is:  tensor(3.6444, device='cuda:0')
current relative error is:  tensor(0.3784, device='cuda:0')
current epoch is:  9250
current loss is:  tensor(3.8841, device='cuda:0')
current relative error is:  tensor(0.3866, device='cuda:0')
current epoch is:  9300
current loss is:  tensor(3.6325, device='cuda:0')
current relative error is:  tensor(0.3652, device='cuda:0')
current epoch is:  9350
current loss is:  tensor(3.7235, device='cuda:0')
current relative error is:  tensor(0.3569, device='cuda:0')
current epoch is:  9400
current loss is:  tensor(3.2159, device='cuda:0')
current relative error is:  tensor(0.3728, device='cuda:0')
current epoch is:  9450
current loss is:  tensor(4.6596, device='cuda:0')
current relative error is:  tensor(0.4191, device='cuda:0')
current epoch is:  9500
current loss is:  tensor(3.1684, device='cuda:0')
current relative error is:  tensor(0.3581, device='cuda:0')
current epoch is:  9550
current loss is:  tensor(6.3043, devic

current epoch is:  12250
current loss is:  tensor(1.8092, device='cuda:0')
current relative error is:  tensor(0.2788, device='cuda:0')
current epoch is:  12300
current loss is:  tensor(2.1591, device='cuda:0')
current relative error is:  tensor(0.3140, device='cuda:0')
current epoch is:  12350
current loss is:  tensor(1.8285, device='cuda:0')
current relative error is:  tensor(0.2973, device='cuda:0')
current epoch is:  12400
current loss is:  tensor(1.9164, device='cuda:0')
current relative error is:  tensor(0.2750, device='cuda:0')
current epoch is:  12450
current loss is:  tensor(3.0537, device='cuda:0')
current relative error is:  tensor(0.3132, device='cuda:0')
current epoch is:  12500
current loss is:  tensor(1.8782, device='cuda:0')
current relative error is:  tensor(0.2650, device='cuda:0')
current epoch is:  12550
current loss is:  tensor(1.9400, device='cuda:0')
current relative error is:  tensor(0.2804, device='cuda:0')
current epoch is:  12600
current loss is:  tensor(2.204

...

current epoch is:  36650
current loss is:  tensor(0.4029, device='cuda:0')
current relative error is:  tensor(0.0823, device='cuda:0')
current epoch is:  36700
current loss is:  tensor(0.7385, device='cuda:0')
current relative error is:  tensor(0.0702, device='cuda:0')
current epoch is:  36750
current loss is:  tensor(0.4119, device='cuda:0')
current relative error is:  tensor(0.0698, device='cuda:0')
current epoch is:  36800
current loss is:  tensor(0.3882, device='cuda:0')
current relative error is:  tensor(0.0777, device='cuda:0')
current epoch is:  36850
current loss is:  tensor(0.3663, device='cuda:0')
current relative error is:  tensor(0.0703, device='cuda:0')
current epoch is:  36900
current loss is:  tensor(0.5397, device='cuda:0')
current relative error is:  tensor(0.0668, device='cuda:0')
current epoch is:  36950
current loss is:  tensor(0.4062, device='cuda:0')
current relative error is:  tensor(0.0817, device='cuda:0')
current epoch is:  37000
current loss is:  tensor(1.126

current epoch is:  39700
current loss is:  tensor(0.5235, device='cuda:0')
current relative error is:  tensor(0.0833, device='cuda:0')
current epoch is:  39750
current loss is:  tensor(0.3192, device='cuda:0')
current relative error is:  tensor(0.0691, device='cuda:0')
current epoch is:  39800
current loss is:  tensor(0.4295, device='cuda:0')
current relative error is:  tensor(0.0659, device='cuda:0')
current epoch is:  39850
current loss is:  tensor(1.7956, device='cuda:0')
current relative error is:  tensor(0.0610, device='cuda:0')
current epoch is:  39900
current loss is:  tensor(0.3280, device='cuda:0')
current relative error is:  tensor(0.0718, device='cuda:0')
current epoch is:  39950
current loss is:  tensor(0.7661, device='cuda:0')
current relative error is:  tensor(0.0849, device='cuda:0')
current epoch is:  40000
current loss is:  tensor(0.3602, device='cuda:0')
current relative error is:  tensor(0.0754, device='cuda:0')
current epoch is:  40050
current loss is:  tensor(0.351

current epoch is:  42750
current loss is:  tensor(0.4998, device='cuda:0')
current relative error is:  tensor(0.0573, device='cuda:0')
current epoch is:  42800
current loss is:  tensor(0.2981, device='cuda:0')
current relative error is:  tensor(0.0666, device='cuda:0')
current epoch is:  42850
current loss is:  tensor(0.6377, device='cuda:0')
current relative error is:  tensor(0.0716, device='cuda:0')
current epoch is:  42900
current loss is:  tensor(0.2906, device='cuda:0')
current relative error is:  tensor(0.0665, device='cuda:0')
current epoch is:  42950
current loss is:  tensor(1.7822, device='cuda:0')
current relative error is:  tensor(0.0944, device='cuda:0')
current epoch is:  43000
current loss is:  tensor(0.5283, device='cuda:0')
current relative error is:  tensor(0.0583, device='cuda:0')
current epoch is:  43050
current loss is:  tensor(1.9157, device='cuda:0')
current relative error is:  tensor(0.1006, device='cuda:0')
current epoch is:  43100
current loss is:  tensor(0.429

current epoch is:  45800
current loss is:  tensor(0.3224, device='cuda:0')
current relative error is:  tensor(0.0592, device='cuda:0')
current epoch is:  45850
current loss is:  tensor(0.5950, device='cuda:0')
current relative error is:  tensor(0.0534, device='cuda:0')
current epoch is:  45900
current loss is:  tensor(0.2678, device='cuda:0')
current relative error is:  tensor(0.0586, device='cuda:0')
current epoch is:  45950
current loss is:  tensor(2.0655, device='cuda:0')
current relative error is:  tensor(0.0912, device='cuda:0')
current epoch is:  46000
current loss is:  tensor(0.3623, device='cuda:0')
current relative error is:  tensor(0.0673, device='cuda:0')
current epoch is:  46050
current loss is:  tensor(0.2732, device='cuda:0')
current relative error is:  tensor(0.0627, device='cuda:0')
current epoch is:  46100
current loss is:  tensor(0.2692, device='cuda:0')
current relative error is:  tensor(0.0581, device='cuda:0')
current epoch is:  46150
current loss is:  tensor(0.720

current epoch is:  48850
current loss is:  tensor(0.8109, device='cuda:0')
current relative error is:  tensor(0.0782, device='cuda:0')
current epoch is:  48900
current loss is:  tensor(0.2396, device='cuda:0')
current relative error is:  tensor(0.0554, device='cuda:0')
current epoch is:  48950
current loss is:  tensor(0.2871, device='cuda:0')
current relative error is:  tensor(0.0698, device='cuda:0')
current epoch is:  49000
current loss is:  tensor(0.2744, device='cuda:0')
current relative error is:  tensor(0.0642, device='cuda:0')
current epoch is:  49050
current loss is:  tensor(0.3393, device='cuda:0')
current relative error is:  tensor(0.0506, device='cuda:0')
current epoch is:  49100
current loss is:  tensor(0.3515, device='cuda:0')
current relative error is:  tensor(0.0595, device='cuda:0')
current epoch is:  49150
current loss is:  tensor(0.4108, device='cuda:0')
current relative error is:  tensor(0.0498, device='cuda:0')
current epoch is:  49200
current loss is:  tensor(0.365

In [18]:
x_test = torch.rand(10, 8)
torch.abs(model(x) - u_ex(x))

tensor([[0.1347],
        [0.0469],
        [0.2349],
        ...,
        [0.0153],
        [0.0472],
        [0.1488]], device='cuda:0', grad_fn=<AbsBackward>)